# 导入相关包

In [1]:
# 导入相关包
import os
import pathlib as pl
import pandas as pd
import numpy as np
import re
from io import StringIO
from datetime import datetime 
import time
from IPython.core.interactiveshell import InteractiveShell
from tqdm.autonotebook import *
import pdfplumber
tqdm.pandas()
InteractiveShell.ast_node_interactivity = "all"
sys.path.append("..")


# PDF解析原始数据 
## 加载数据并采用pdfplumber抽取PDF中的文字和表格


In [2]:
# 数据准备(train_output文件中格式有点问题，需要提前用excel或者wps打开然后另存为excel文件)
train_outputs = pd.read_excel('../datasets/train_output.xlsx')

# 获取pdf中文字和表格
def extract_pdf_content(pdf_path):
    text_list = []
    table_list = []
    with pdfplumber.open(pdf_path) as pdf:
        for index_page in np.arange(0, len(pdf.pages), 1):
            # 读取多页
            page = pdf.pages[index_page]   # 第n页的信息
            text = page.extract_text()
            text_list.append(text)
            table = page.extract_tables()
            for t in table:
                table_list.append(t)
    return text_list, table_list

def get_dir_file(path):
    '''
    输入文件夹位置，输出整理好的dataframe
    '''
    path_list = os.listdir(path)
    id_list = []
    file_path_list = []
    text_list = []
    table_list = []
    for i in tqdm(path_list):
        if '.PDF' in i:
            file_path = path + i
            id_list.append(int(i.split('.')[0]))
            file_path_list.append(file_path)
            try:
                text_temp, table_temp = extract_pdf_content(file_path)
            except Exception:
                print('此pdf无法读取')
                text_temp, table_temp = [], []
            text_list.append(text_temp)
            table_list.append(table_temp)
            
    df = pd.DataFrame()
    df['sample_id'] = id_list
    df['file_path'] = file_path_list
    df['text'] = text_list
    df['tabel'] = table_list
    df = df.sort_values('sample_id')
    return df

# 文件处理太慢，可持续化保存文件
train_path = '../datasets/train.csv'
if os.path.exists(train_path):
    train_df = pd.read_csv(train_path)
else:
    train_df = get_dir_file('datasets/train_data/')
    train_df.to_csv(train_path,index=False)
    train_df = pd.read_csv(train_path)

test_path =  '../datasets/test.csv'
if os.path.exists(test_path):
    test_df = pd.read_csv(test_path)
else:
    test_df = get_dir_file('datasets/test_data/')
    test_df.to_csv(test_path,index=False)
    test_df = pd.read_csv(test_path)

train_outputs.head(2)
train_df.head(2)
test_df.head(2)

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1,2019-03-27,汇聚金1号,中融国际信托有限公司,信托,10000.0,2019-03-27,2019-09-23,180天,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25
1,1,2019-03-27,招商银行步步生金8699,招商银行,银行理财产品,200.0,2019-03-27,NaT,NaN,自有资金,恒生电子股份有限公司,公司本身,否,2019-04-25


,sample_id,file_path,text,tabel
0,1,datasets/train_data/1.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."
1,2,datasets/train_data/2.PDF,[' ...,"[[['', None, None, '', None, None, '', None, N..."


,sample_id,file_path,text,tabel
0,11188,datasets/test_data/11188.PDF,['北京京西文化旅游股份有限公司监事会\n \n \n关于使用部分闲置募集资金购买理财产品的...,[]
1,11189,datasets/test_data/11189.PDF,['北京京西文化旅游股份有限公司 \n监事会关于使用部分自有资金购买理财产品的意见 \n根据...,[]


In [3]:
# 构造训练集验证集
train_df = train_df.sample(frac=1, random_state=1017)
val_df = train_df[:1800]
train_df = train_df[1800:]

# 数据处理
## 抽取整体数据（一个sampleid内此字段内容都相同）
## 公告时间，实际购买公司

#### 1.抽取公告时间

In [4]:
# 首先针对任务抽取时间（每个时间跟每个id是一一对应的）
# 要不是取第一个时间，要不就是取最后一个时间（或者时间加一）这里可以建立一个模型预测
# base这里面直接取最后一个时间作为发布日期

CN_NUM = {
    u'〇': 0, u'一': 1, u'二': 2, u'三': 3,
    u'四': 4, u'五': 5, u'六': 6, u'七': 7,
    u'八': 8, u'九': 9, u'零': 0, u'壹': 1,
    u'贰': 2, u'叁': 3, u'肆': 4, u'伍': 5,
    u'陆': 6, u'柒': 7, u'捌': 8, u'玖': 9,
    u'貮': 2, u'两': 2,
}


def get_put_time_from_text(row):
    row = row.replace(' ', '').replace('\\n', '')
    for key in CN_NUM:
        row = row.replace(key, str(CN_NUM[key]))   
    r = row.replace("年", "-").replace("月", "-").replace("日", " ").replace("/", "-").strip()
    regex = "(\d{4}-\d{1,2}-\d{1,2})"
    r = re.findall(regex, r)
    if len(r)==0:
        return np.nan
    time_str = r[-1]
    first = time_str.split('-')[0]
    second = time_str.split('-')[1]
    last = time_str.split('-')[-1]
    second = str.zfill(second, 2)
    last = str.zfill(last, 2)
    r = '-'.join([first, second, last])
    return r

val_result = pd.DataFrame()
val_result['sample_id'] = val_df['sample_id']
val_result['predict_time'] = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
test_gg = train_outputs.groupby('sample_id').apply(lambda row:list(row['公告日期'])[0]).reset_index()
test_gg.columns = ['sample_id', 'time']
val_result = pd.merge(val_result, test_gg, on='sample_id', how='left')

# 判断验证集的准确率
np.sum(val_result['predict_time'].astype(str) == val_result['time'].astype(str))/len(val_result)

val_time = val_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)
# test_time = test_df.progress_apply(lambda row: get_put_time_from_text(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 2006.69it/s]


0.4583333333333333

100%|██████████| 1800/1800 [00:00<00:00, 2821.31it/s]


#### 2.抽取实际购买公司

In [5]:
# 抽取购买公司
# 前几句话出现
# 将其按照\\n 和空格切割
def get_gm(row):
    result = re.split('[\\\\n ]',row)
    for i in result:
        if '公司' in i:
            return i

val_gm = val_df.progress_apply(lambda row:get_gm(row['text']), axis=1)
# test_gm = test_df.progress_apply(lambda row:get_gm(row['text']), axis=1)

100%|██████████| 1800/1800 [00:00<00:00, 2593.67it/s]


#### 3.清洗提取出来的tabel数据，主要是清洗掉有问题的列 
# 重写清洗方法

# 将table转换格式以及处理
def deal_tabel(row):
    row = eval(row)
    if len(row)==0:
        return []
    else:
        new_row = []
        for i in row:
            for d in i:
                new_temp = []
                for h in d:
                    # 这里处理空数据或者错误的数据
                    h = str(h).replace('None', '').replace('\n','').replace(' ', '')                    
                    if h=='':
                        continue
                    if h=='.':
                        continue
                    if h=='/':
                        continue
                    new_temp.append(h)
                new_row.append(new_temp)
        # 这里判断是否构成一个完整得认购数据(通过一个list进行判断)
        new_new_row = []
        for i in new_row:
            if len(i) == 0:
                continue
            elif len(i) <= 4:
                continue
            else:
                new_new_row.append(i)
        return new_new_row
# train_df_tabel = train_df['tabel'].progress_apply(lambda row:deal_tabel(row))
val_df_tabel = val_df['tabel'].progress_apply(lambda row:deal_tabel(row))

# test_df_tabel = test_df['tabel'].progress_apply(lambda row:deal_tabel(row))

# 将table转换格式以及处理
def deal_tabel(row):
    row = eval(row)
    if len(row)==0:
        return []
    else:
        new_row = []
        for i in row:
            for d in i:
                new_temp = []
                for h in d:
                    # 这里处理空数据或者错误的数据
                    h = str(h).replace('None', '').replace('\n','').replace(' ', '')                    
                    if h=='':
                        continue
                    if h=='.':
                        continue
                    if h=='/':
                        continue
                    new_temp.append(h)
                new_row.append(new_temp)
        # 这里判断是否构成一个完整得认购数据(通过一个list进行判断)
        new_new_row = []
        for i in new_row:
            if len(i) == 0:
                continue
            elif len(i) <= 4:
                continue
            else:
                new_new_row.append(i)
        return new_new_row
# train_df_tabel = train_df['tabel'].progress_apply(lambda row:deal_tabel(row))
val_df_tabel = val_df['tabel'].progress_apply(lambda row:deal_tabel(row))

# test_df_tabel = test_df['tabel'].progress_apply(lambda row:deal_tabel(row))

In [6]:
# tmp_table=pd.concat([val_df["sample_id"],pd.DataFrame(val_df_tabel)],axis=1)
# tmp_table["len_table"]=tmp_table["tabel"].apply(lambda x:[len(i) for i in x])
# # tmp_table.to_excel("table矩阵检验1.xlsx")

NameError: name 'val_df_tabel' is not defined

#大量PDF的理财购买信息不在table中，而是在text中，此段为判断理财信息是否只需要从text中提取

In [7]:
title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","到期","截至"]


def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
    for item in title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])
    
    # for item in title_list:
    for item in s_title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(2)
            text_end_iter_list.append(i.span(0)[1])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\n","")
    title_df=get_title(text)
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    index=0
    neg_index=[]
    for title_des in title_1_df[0].values:
        for item in title_neg_words:
            if re.search(item,title_des) is not None:
                neg_index.append(index)
                break
        index+=1


    return title_1_df.drop(neg_index)
    # print(title_list)

judge_title_result=None


for sample_id,text in tqdm(val_df[["sample_id","text"]].values):
    # print(sample_id)
    # print(text)
    judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])

# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)


# is_from_text(val_df[val_df["sample_id"]==930]["sample_id"].iloc[0],val_df[val_df["sample_id"]==930]["text"].iloc[0])
# is_from_text(val_df[val_df["sample_id"]==125]["text"].iloc[0])

100%|██████████| 1800/1800 [00:27<00:00, 65.83it/s]


In [8]:
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==35]["tabel"].iloc[0])
# # tabel_tmp_value_list=[]
# table_value_df=pd.DataFrame(tabel_tmp_value)
val_df.loc[1419].shape[0]
# tmp2=val_df.head(51).index
# list(tmp1)
# np.array_equal(tmp1,tmp2)
# set(tmp2).issubset(tmp1)
# set(tmp2).difference(tmp1)

4

In [9]:
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==4333]["tabel"].iloc[0])
# tabel_tmp_value=eval(val_df[val_df["sample_id"]==35]["tabel"].iloc[0])
# tabel_tmp_value_list=[]
# table_value_df=pd.DataFrame(tabel_tmp_value[0])
# table_value_df.head(20)
# for item in tabel_tmp_value:

column_name_judge_word=["民币）","民币)","万元)","元)","万元）","元）","%)","%）","金额"]


#寻找（字段）所在的理论最后一行
def find_amt(df):
    global column_name_judge_word
    max_index=-1
    df=df.head(7)
    for index in range(df.shape[0]):
        for item in column_name_judge_word:
            flag=[]
            flag_num=[]
            df.loc[index].map(lambda x:flag_num.append(re.match("[ ]*?\d",x)))
            df.loc[index].map(lambda x:flag.append((item in x)))
            for i in flag_num:
                if(i is not None):
                    return max_index
            if True in flag:
                max_index=index
    return max_index

column_name_judge_word=["民币）","民币)","万元)","元)","万元）","元）","%)","%）","金额"]


#寻找单个理财产品所在的理论最后一行
def find_product(df):
    global column_name_judge_word
    max_index=-1
    df=df.head(7)
    for index in range(df.shape[0]):
        for item in column_name_judge_word:
            flag=[]
            flag_num=[]
            df.loc[index].map(lambda x:flag_num.append(re.match("[ ]*?\d",x)))
            df.loc[index].map(lambda x:flag.append((item in x)))
            for i in flag_num:
                if(i is not None):
                    return max_index
            if True in flag:
                max_index=index
    return max_index

def get_valid_columns_num(df):
    df=df.T.dropna(axis=0).apply(lambda x:x.replace(" ",""))
    return df[df!=""].shape[0]

def get_valid_columns_index(df):
    df=df.T.dropna(axis=0).apply(lambda x:x.replace(" ",""))
    return df[df!=""].index

def field_location_optimization(columns_list,df):
    if(df.shape[1]!=columns_list.shape[0]):
        return None
    max_columns_num=get_valid_columns_num(columns_list)
    if(df.shape[0]==0):
        return df
    valid_clomuns_index=list(get_valid_columns_index(columns_list))
    # print(valid_clomuns_index)
    result_df=df.head(0)
    # print(result_df.shape)
    # print(df)
    for index in range(df.shape[0]):
        tmp_row=df.loc[index]
        tmp_index=get_valid_columns_index(tmp_row)
        # print(list(tmp_index))
        if set(tmp_index).issubset(valid_clomuns_index):
            pass
        else:
            difference_list=list(set(tmp_index).difference(valid_clomuns_index))
            
            location_list=[]
            tmp_difference_list=difference_list.copy()
            for item in difference_list:
                location=item
                #前后浮动位移（可优化）
                for i in range(1,3):
                    location=(item-i)
                    if location in valid_clomuns_index and location not in tmp_difference_list:
                        location_list.append(location)
                        tmp_difference_list.remove(item)
                        tmp_difference_list.append(location)
                        break
                    location=(item+i)
                    if location in valid_clomuns_index and location not in tmp_difference_list:
                        location_list.append(location)
                        tmp_difference_list.remove(item)
                        tmp_difference_list.append(location)
                        break
            result_location=list(df.columns)
            # print(location_list)
            # print(difference_list)
            # print("---------------------")
            for i in range(len(location_list)):
                result_location[difference_list[i]]=location_list[i]
                result_location[location_list[i]]=difference_list[i]
            # print(tmp_row)
            tmp_row=tmp_row.iloc[result_location].reset_index(drop=True)
            # print(tmp_row)
            # print(result_location)
        result_df=tmp_row.to_frame().T if result_df is None else pd.concat([result_df,tmp_row.to_frame().T])
    # print(np.array_equal(get_valid_columns_index(columns_list),get_valid_columns_index(result_df)))
    # print(list(set(get_valid_columns_index(columns_list)).difference(result_df)))
    # print(result_df)
    # print(result_df.shape)
    return result_df
def row_combine(sample_id,pdf_table):
# 将字符串转化成多维列表
    pdf_table=eval(pdf_table)

    table_result=[]
    start_rows_list=[]
    first_line_list=[]
    product_df_list=[]
    for item in pdf_table:
        #考虑加入一个判定表格位置的方法（暂缺）

        tmp_table_df=pd.DataFrame(item)
        tmp_table_df=tmp_table_df.fillna("").applymap(lambda x: x.replace("\n",""))
        #清除全空行
        drop_index_list=[]
        for index in range(tmp_table_df.shape[0]):
            judge=[]
            noshow=tmp_table_df.loc[index].map(lambda x:judge.append(x==""))
            if(False not in judge):
                drop_index_list.append(index)
        
        tmp_table_df=tmp_table_df.drop(drop_index_list).reset_index(drop=True)

        base_row=find_amt(tmp_table_df)
        if base_row==-1:
            if(len(table_result)>0):
                product_df=field_location_optimization(table_result[len(table_result)-1],tmp_table_df.loc[:].reset_index(drop=True))
                if(product_df is not None):
                    len_product_df_list=len(product_df_list)-1
                    product_df_list[len_product_df_list]=pd.concat([product_df_list[len_product_df_list],product_df])
                    # print(product_df.shape)
                else:
                    pass
            continue

        tmp_row=None
        valid_columns_nums=[]
        len_df=None
        for index in range(0,base_row+1):
            len_list=tmp_table_df.loc[index].map(lambda x:len(x))
            for i in range(len(len_list)):
                if len_list[i]==0 and len_df is not None:
                    len_list[i]=len_df.tail(1)[i]
            len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T])
            row_result=tmp_table_df.loc[index].map(lambda x:re.sub(r"$[ ]+?","",re.sub(r"^[ ]+?","",x)))
            tmp_row= row_result if tmp_row is None else tmp_row+row_result
            # valid_columns_nums.append(get_valid_columns_num(tmp_row))
        
        start_row=base_row+1

        for index in range(base_row+1,tmp_table_df.shape[0]):
            len_list=tmp_table_df.loc[index].map(lambda x:len(x))
            len_judge=pd.DataFrame(len_list).T-len_df.loc[len_df.shape[0]-1]
            len_judge=len_judge.T
            if(len_judge[len_judge>0].shape[0]>0):
                break
            start_row+=1
            for i in range(len(len_list)):
                if len_list[i]==0 and len_df is not None:
                    len_list[i]=len_df.tail(1)[i]
            len_df=pd.DataFrame(len_list).T if len_df is None else pd.concat([len_df,pd.DataFrame(len_list).T])
            row_result=tmp_table_df.loc[index].map(lambda x:re.sub(r"$[ ]+?","",re.sub(r"^[ ]+?","",x)))
            tmp_row= row_result if tmp_row is None else tmp_row+row_result
            # valid_columns_nums.append(get_valid_columns_num(tmp_row))
        
        columns_list=tmp_row.dropna().map(lambda x:x.replace(" ",""))
        
        #处理错列问题
        product_df=field_location_optimization(columns_list,tmp_table_df.loc[start_row:].reset_index(drop=True))
        
        first_line=[]
        tmp_table_df.head(2).applymap(lambda x:first_line.append(x))
        first_line_result=[]
        for i in first_line:
            i=re.sub(r"^[ ]*?","",i)
            i=re.sub(r"$[ ]*?","",i)
            # i=re.sub(r"[ ]*?","",i)
            if i != "":
                first_line_result.append(i)
        table_result.append(columns_list)
        first_line_list.append(first_line_result)
        start_rows_list.append(start_row)
        product_df_list.append(product_df)
        # print(len(table_result))
        # print(len(first_line_list))
        # print(len(start_rows_list))
    return table_result,start_rows_list,first_line_list,product_df_list


tmp_table_column={}
tmp_table_column["sample_id"]=[]
tmp_table_column["columns"]=[]
tmp_table_column["start_row"]=[]
tmp_table_column["first_line"]=[]
tmp_table_column["product_df"]=[]
for sample_id,tabel in tqdm(val_df[["sample_id","tabel"]].head(1800).values):##修改使用的数据集
    table_result,start_rows_list,first_line_list,product_df_list=row_combine(sample_id,tabel)
    for i in range(len(start_rows_list)):
        valid_columns_index=get_valid_columns_index(table_result[i])
        tmp_table_column["sample_id"].append(sample_id)
        tmp_table_column["columns"].append(table_result[i].iloc[valid_columns_index].T.reset_index(drop=True).T)
        tmp_table_column["start_row"].append(start_rows_list[i])
        tmp_table_column["first_line"].append(first_line_list[i])
        tmp_table_column["product_df"].append(product_df_list[i].iloc[:,valid_columns_index].T.reset_index(drop=True).T)
tmp_table_column=pd.DataFrame(tmp_table_column)


# tmp_table_column[["columns","start_row","first_line"]]=val_df["tabel"].head(5).apply(lambda x:row_combine(x))


# tmp=val_df.iloc[177,:][["sample_id","tabel"]]
# table_result,start_rows_list,first_line_list,product_df_list=row_combine(tmp["sample_id"],tmp["tabel"])
# table_result
# product_df_list

100%|██████████| 1800/1800 [07:12<00:00,  4.17it/s]


#fuzz裁剪，待优化

In [11]:
# from fuzzywuzzy import fuzz
# index=0
# invalid_list=[]
# for sample_id,first_line in tqdm(tmp_table_column[["sample_id","first_line"]].values):
#     current_judge_title=judge_title_result[judge_title_result["sample_id"]==sample_id]
#     judge_flag=[]
#     for item in first_line:
#         judge_flag=[]
#         for i in current_judge_title[4].values:
#             judge_flag.append(fuzz.partial_token_sort_ratio(item,i))
#         if(np.mean(judge_flag)>0):
#             print(np.mean(judge_flag))
#             invalid_list.append(index)
#             break
#     index+=1
# invalid_list

#单个理财产品行划分

In [57]:

from src.time_extractor import TimeFinder
import datetime
a='2018年6月24日，2018年6月24日'
t = TimeFinder()
time_all = t.find_time(a)
print(time_all)

a="20.01万元"
re.sub("[^0-9.]","",a)
a="1"
a=a+"天"
a=["1","2","3"]
b=("").join(i for i in a)

len(pd.Series(a).shape)


['2018-06-24', '2018-06-24']


'20.01'

1

In [79]:
from fuzzywuzzy import fuzz
from src.time_extractor import TimeFinder
import datetime


#寻找居中列，即只占一行的列
def find_min_columns(df):
    count_list=[]
    result=[]
    df.copy().apply(lambda x:count_list.append(get_valid_columns_num(x.T)),axis=1)
    min_count=np.min(count_list)
    for item in count_list:
        if item ==min_count:
            result.append(item)
    return result
def judge_time_exist(df):
    judge_flag=[]
    t = TimeFinder()
    df.map(lambda x:judge_flag.append(t.find_time(x) is not None))
    if True not in judge_flag:
        return False
    else:
        return True
def get_each_product_row(columns_list,df):
    global column_neg_words
    df=df.applymap(lambda x:str(x).replace(" ",""))
    max_valid_index_num=len(columns_list)

    #字段个数不足，跳过
    if(columns_list.shape[0]<=4):
        # print(1)
        return None

    #存在敏感词，即为无效答案，跳过
    judge_flag=[]    
    for word in column_neg_words:
        columns_list.map(lambda x:judge_flag.append(fuzz.partial_ratio(word,x)==100))
    if(True in judge_flag):
        # print(2)
        return None
    sum_rows=df.head(0)
    each_row=None

    #获取居中列
    min_columns_list=find_min_columns(df)

    for index in range(df.shape[0]):
        tmp_row=df.loc[index]
        # print(tmp_row.shape)
        tmp_valid_num=get_valid_columns_num(tmp_row)

        if tmp_valid_num==max_valid_index_num:#第一种情况，列完整
            #判断是否存在时间
            if(judge_time_exist(tmp_row)):
                each_row=None
                sum_rows=pd.concat([sum_rows,tmp_row.to_frame().T])
                continue
            else:#不存在时间则视为不完整,加入each_row之后判断是否存在时间，存在即完整
                if each_row is None:
                    each_row=tmp_row
                else:
                    each_row=each_row+tmp_row
                    if(judge_time_exist(each_row)):
                        each_row=None
                        sum_rows=pd.concat([sum_rows,tmp_row.to_frame().T])
                        continue
                    else:
                        continue
                
        elif(tmp_valid_num==1 and sum_rows.shape[0]==0):#第二种情况，列不完整,且长度为1,且sum_row无内容，认为是少数的错误字段遗留数据，舍弃
            continue
        elif((index+1)!=df.shape[0]):#第三种情况，列不完整，需要多行拼加
            #寻找目前剩下的矩阵中与当前行以外存在值的最浅的居中列
            
            start_row=index
            remain_df=df.iloc[start_row:,min_columns_list]
            middle_index=index+1
            middle_column=min_columns_list[0]
            for i in list(remain_df.index):
                valid_columns_index=get_valid_columns_index(remain_df.loc[i])
                if(len(valid_columns_index))>0:
                    middle_index=i
                    middle_column=valid_columns_index[0]
                    break
            
            start_row=index
            end_row=middle_index*2-index if middle_index*2-index<=df.shape[0] else df.shape[0]
            for i in range(start_row,end_row):
                if(get_valid_columns_num(df.loc[i])==max_valid_index_num):#强制中断条件
                    break
                index=i
                each_row=df.loc[i] if each_row is None else each_row+df.loc[i]

                if(judge_time_exist(each_row)):
                    each_row=None
                    sum_rows=pd.concat([sum_rows,tmp_row.to_frame().T])
                    continue
                else:
                    each_row=None
                    continue
    # print(sum_rows)
    return sum_rows


column_neg_words=["实际收回","收回","赎回","实际获得","实际损益","收益情况","投资盈亏","投资收益","理财盈亏","理财收益","盈亏","收益（元","收益(元","收益(万元","收益（万元","到期收益","到期收","是否到","是否已","目前状","到期情","到息情"]
sum_product_df=[]

index=0
index_list=[]
for columns_list,product_df in tqdm(tmp_table_column[["columns","product_df"]].values):
    # product_df
    each_sum_rows=get_each_product_row(columns_list,product_df.reset_index(drop=True))
    if(each_sum_rows is not None):
        sum_product_df.append(each_sum_rows)
        index_list.append(index)
    index+=1

result_matrix=tmp_table_column.iloc[index_list,:]
result_matrix["product_df"]=sum_product_df


100%|██████████| 2162/2162 [10:47<00:00,  3.34it/s]


In [95]:
sample_id=3750
tabel=val_df[val_df["sample_id"]==sample_id]["tabel"].iloc[0]
table_result,start_rows_list,first_line_list,product_df_list=row_combine(sample_id,tabel)
table_result[0]
product_df_list_ele=product_df_list[0]
product_df_list_ele
# each_sum_rows=get_each_product_row(table_result[0],product_df_list[0].reset_index(drop=True))
# each_sum_rows

0                 
1             产品名称
2                 
3                 
4             产品类型
5                 
6                 
7          产品期限（天）
8                 
9                 
10              币种
11                
12                
13        认购金额（万元）
14                
15                
16    预期年化最高收益率（%）
17                
18                
19             起息日
20                
21                
22             到息日
23                
24                
25            投资范围
26                
27                
28            资金来源
29                
dtype: object

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,,,,,,,,,,,...,,,,,,本理财产品主要投资于债券、,,,,
1,,江苏银行“聚宝,,,,,,,,,...,,,,,,,,,昆山维信诺科,
2,,,,,保本浮动,,,,,,...,,,,,,货币市场工具等各类高流动,,,,
3,,财富天添开鑫”,,,,,,不定期,,,...,,,---,,,,,,技有限公司自,
4,,,,,收益型,,,,,,...,,,,,,性资产，符合监管要求的各类,,,,
5,,开放式理财产品,,,,,,,,,...,,,,,,,,,有闲置资金,
6,,,,,,,,,,,...,,,,,,债权类资产及其组合。,,,,
7,,,,,,,,,,,...,,,,,,本结构性存款产品资金存放,,,,
8,,结构性存款SD南,,,,,,,,,...,,,,,,于广东南粤银行，由广东南粤,,,云谷（固安）,
9,,,,,保本浮动,,,,,,...,,,,,,,,,,


#### 4.抽取的是单独的数据包含
#### 起息日，到息日， 金额，认购日期，产品发行方，理财产品

In [80]:
temp_single={}
temp_single['认购日期'] = []
temp_single['产品起息日'] = []
temp_single['产品到息日'] = []
temp_single['产品期限'] = []
temp_single['认购金额(万元)'] = []
temp_single['产品发行方名称'] = []
temp_single['理财产品名称'] = []
temp_single['sample_id'] = []

            
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

def judge_type(columns):
    type_index=[]#1:产品名,2:金额,3:发行方，4:期限
    columns=columns.map(lambda x:x.replace("（","(").replace("）",")"))
    product_name_pos_words=["产品名称","产品名册","产品名","理财产品","项目名","回购名","回购品","标的名","金融产","投资项"]#"存款种类","基金类型"#不能为空
    # product_name_neg_words=["编号","代码"]
    amt_pos_words=["存款金","认购金","投资金","投入金","受托金","理财金","金额","（元","(元","(万元","（万元","(亿元","（亿元","人民币","投资规","认购规","存款规","投入规","理财规"]
    counter_name_pos_words=["受托方","银行机","机构名","合作方名","合作银","合作机","受托人","发行主","签约方","协议方","受托机","受托银","认购银","签约银","签约机","协议机","发生主","存放银","存款银","存款机","存放机","购买银","购买机","管理人","管理银","管理机","银行名","发行机","发行主","发行人","对手方","开户银","开户行","开户机"]#可以为空
    time_length_pos_words=["期限","(天)","持有时间"]
    for words in product_name_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        # columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==0))
        if True in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==0):
        for words in ["种类","类型","类别"]:
            judge_flag=[]
            columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
            # columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==0))
            if True  in judge_flag:
                type_index.append(judge_flag.index(True))
                break
        if(len(type_index)==0):
            type_index.append(-1)
        

    for words in amt_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        if True  in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==1):
        type_index.append(-1)
    
    for words in counter_name_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        if True  in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==2):
        type_index.append(-1)
    
    for words in time_length_pos_words:
        judge_flag=[]
        columns.map(lambda x:judge_flag.append(fuzz.partial_ratio(words,x)==100))
        if True  in judge_flag:
            type_index.append(judge_flag.index(True))
            break
    if(len(type_index)==3):
        type_index.append(-1)
    
    
    return type_index

for sample_id,columns,product_df in tqdm(result_matrix[["sample_id","columns","product_df"]].values):
    
    type_index=judge_type(columns)
    # product_df
    # columns
    # type_index
    for index in product_df.index:
        tmp_df=product_df.loc[index]
        if(len(tmp_df.shape) ==2 ):
            tmp_df=tmp_df.reset_index(drop=True).loc[0]
        product_name=np.nan
        amt=np.nan
        counter_name=np.nan
        pur_dt=np.nan
        val_dt=np.nan
        coupon_dt=np.nan
        time_limit=np.nan
        #产品名
        if(type_index[0]!=-1):
            product_name=tmp_df.loc[type_index[0]]

        #金额
        if(type_index[1]!=-1):
            amt=tmp_df.loc[type_index[1]].replace("（","").replace("）","").replace("(","").replace("(","").replace("元","").replace("圆","")
            type_amt=0
            if("万" in amt or "万" in columns.loc[type_index[1]]):
                type_amt=1
            if("亿" in amt or "亿" in columns.loc[type_index[1]]):
                type_amt=2
            amt=re.sub("[^0-9.]","",amt)
            if(is_number(amt)):
                amt=float(amt)
                if(type_amt==0):
                    amt/=10000
                if(type_amt==2):
                    amt*=10000
        else:
            value_list=list(tmp_df)
            for item in value_list:
                tmp=str(item).replace("（","").replace("）","").replace("(","").replace("(","").replace("元","").replace("圆","").replace("亿","").replace("万","")
                if(is_number(tmp)):
                    amt=item
                    type_amt=0
                else:
                    continue
                if("万" in amt ):
                    type_amt=1
                if("亿" in amt ):
                    type_amt=2
                amt=re.sub("[^0-9.]","",amt)
                if(is_number(amt)):
                    amt=float(amt)
                    print(amt)
                    if(type_amt==0):
                        amt/=10000
                    if(type_amt==2):
                        amt*=10000
                if(amt !="" or amt!=np.nan):
                    break
        #发行方
        if(type_index[2]!=-1):
            counter_name=tmp_df.loc[type_index[2]]
        
        #期限
        if(type_index[3]!=-1):
            time_limit=str(tmp_df.loc[type_index[3]]).replace("（","").replace("）","").replace("(","").replace("(","")
            if("天" not in time_limit):
                time_limit=time_limit + "天"
        #三个日期
        # tmp_df
        value_list=[]
        noshow=tmp_df.map(lambda x:value_list.append(str(x)))
        sum_value=("").join(i for i in value_list)
        t = TimeFinder()
        time_all = t.find_time(sum_value)
        if(time_all is not None):
            if(len(time_all)==1):
                pur_dt=time_all[0]
                val_dt=pur_dt
            elif(len(time_all)==2):
                pur_dt=time_all[0]
                val_dt=pur_dt
                coupon_dt = time_all[1]
                try:
                    # 相减
                    if(type_index[3]!=-1):
                        d1 = datetime.datetime.strptime(val_dt, '%Y-%m-%d')
                        d2 = datetime.datetime.strptime(coupon_dt, '%Y-%m-%d')
                        d = d2 - d1
                        time_limit = str(d.days) + '天'
                except Exception:
                    val_dt = np.nan
                    time_limit = np.nan
        temp_single['认购日期'].append(pur_dt)
        temp_single['产品起息日'].append(val_dt)
        temp_single['产品到息日'].append(coupon_dt)
        temp_single['产品期限'] .append(time_limit)
        temp_single['认购金额(万元)'].append(amt)
        temp_single['产品发行方名称'] .append(counter_name)
        temp_single['理财产品名称'] .append(product_name)
        temp_single['sample_id'].append(sample_id)
    
temp_single=pd.DataFrame(temp_single)

23%|██▎       | 235/1002 [01:05<04:49,  2.65it/s]2.7
2.7
2.7
3.0
3.0
2.75
3.25
2.6
2.6
2.7
2.7
 24%|██▎       | 236/1002 [01:05<05:36,  2.28it/s]2.7
2.7
100%|██████████| 1002/1002 [04:49<00:00,  3.46it/s]


In [81]:
temp_single

,认购日期,产品起息日,产品到息日,产品期限,认购金额(万元),产品发行方名称,理财产品名称,sample_id
0,2017-08-28,2017-08-28,2017-11-27,91天,10000,NaN,银行,1806
1,2019-04-16,2019-04-16,2019-07-18,93天,4000,中国银行股份有限公司厦门杏林支行,中银平稳理财计划-智荟系列,3581
2,2019-04-16,2019-04-16,2019-07-16,91天,3000,厦门银行股份有限公司杏林支行,结构性存款,3581
3,2019-04-17,2019-04-17,2019-07-15,89天,3000,兴证证券资产管理有限公司,兴证资管鑫利5号集合资产管理计划,3581
4,2019-04-16,2019-04-16,2019-07-16,91天,10000,兴业银行股份有限,兴业银行“金雪球-优悦”,3581
...,...,...,...,...,...,...,...,...
7072,NaN,NaN,NaN,回天,,,,193
7073,NaN,NaN,NaN,回天,,,,193
7074,2018-05-08,2018-05-08,NaN,天,800,中国农业银行,安心快线天天利滚利第2期,193
7075,NaN,NaN,NaN,回天,,,,193


title_num_char=["一、","二、","三、","四、","五、","六、","七、","八、","九、","十、","十一、","十二、","十三、","十四、","十五、"]
s_title_num_char=["（一）","（二）","（三）","（四）","（五）","（六）","（七）","（八）","（九）","（十）","（十一）","（十二）","（十三）","（十四）","（十五）"]
s_title_num_char.extend(["[(]一[)]","[(]二[)]","[(]三[)]","[(]四[)]","[(]五[)]","[(]六[)]","[(]七[)]","[(]八[)]","[(]九[)]","[(]十[)]","[(]十一[)]","[(]十二[)]","[(]十三[)]","[(]十四[)]","[(]十五[)]"])

title_pos_words=[]
title_neg_words=["备查","日前","过去","履行","审批","程序","风险","措施","影响","累计","赎回","到期"]


def get_title(text):
    global title_num_char
    title_list=[]
    title_type_list=[]
    text_start_iter_list=[]
    text_end_iter_list=[]
    for item in title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(1)
            text_end_iter_list.append(i.span(0)[1])
    
    # for item in title_list:
    for item in s_title_num_char:
        pattern = re.compile(item+r"[ ]*?[^ ]+?[ ]")
        tmp=pattern.finditer(text)
        for i in tmp:
            title_list.append(i.group())
            text_start_iter_list.append(i.span(0)[0])
            title_type_list.append(2)
            text_end_iter_list.append(i.span(0)[1])

    title_list.append("引言")
    title_type_list.append(1)
    text_start_iter_list.append(0)
    text_end_iter_list.append(0)

    result_df=pd.DataFrame([title_list,title_type_list,text_start_iter_list,text_end_iter_list]).T.sort_values(by=2).reset_index(drop=True)
    # print(result_df)
    return result_df

def get_title_text(text,title_df):
    # print(title_df)
    title_1_df=title_df[title_df[1]==1]
    text_iter_list=[]
    text_list=[]
    # print(title_1_df)
    for iter1,iter2 in title_1_df[[2,3]].values:
        # print(iter1)
        if(len(text_iter_list)!=0):
            text_iter_list.append(iter1)
        text_iter_list.append(iter2)
    # text_iter_list.append(text_iter_list[len(text_iter_list)-1])
    text_iter_list.append(len(text))
    for index in range(int(len(text_iter_list)/2)):
        text_list.append(text[text_iter_list[2*index]:text_iter_list[2*index+1]])
    
    title_1_df[4]=text_list

    return title_1_df.reset_index(drop=True)

from fuzzywuzzy import fuzz
def judge_title(sample_id=0,text=r"test\n"):
    # print(text)
    text=text.replace(r"\n","")
    title_df=get_title(text)
    title_df["sample_id"]=[sample_id for x in range(title_df.shape[0])]
    # print(title_df)
    title_1_df=get_title_text(text,title_df)[["sample_id",0,1,2,3,4]]

    

    global val_df
    global train_outputs
    val_true_name=train_outputs[train_outputs["sample_id"]==sample_id]["理财产品名称"]
    
    # print(title_1_df)
    title_1_df["匹配分数"]=0
    for item in val_true_name:
        title_1_df["匹配分数"]=title_1_df["匹配分数"]+title_1_df[4].apply(lambda x:fuzz.partial_ratio(x,str(item)))
    
    max_score=np.max(title_1_df["匹配分数"])
    title_1_df["是否为信息来源"]=title_1_df["匹配分数"].apply(lambda x:0 if x < max_score else 1)
    # print(title_1_df)


    return title_1_df
    # print(title_list)

judge_title_result=None


# for sample_id,text in tqdm(train_df[["sample_id","text"]].values):
#     # print(sample_id)
#     # print(text)
#     judge_title_result= judge_title(sample_id,text) if judge_title_result is None else pd.concat([judge_title_result,judge_title(sample_id,text)])

# judge_title_result.to_excel("训练集段落标题分类结果.xlsx",index=None)


# is_from_text(val_df[val_df["sample_id"]==930]["sample_id"].iloc[0],val_df[val_df["sample_id"]==930]["text"].iloc[0])
# is_from_text(val_df[val_df["sample_id"]==125]["text"].iloc[0])

judge_title(1067,train_df[train_df["sample_id"]==1067]["text"].iloc[0])

# 直接提取时间
# 如果出现两个时间第一个就是起息日，第二个就是到期日
# 如果出现一个时间就是起息日
# 出现的第一个money就是最后的金额
# 从这里面抽取所有序列
# 这里认为有逗号出现的就是money

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
    return False

from src.time_extractor import TimeFinder
import datetime
def get_list_data(df):
    df = list(df)
    new_df = []
    for i in tqdm(df):
        temp_df = []
        for h in i:
            new_h = []
            for digital in h:
                if ',' in digital:
                    # 这里也是为了统一数据有些是用元，有些是用万元
                    try:
                        ttt = float(digital.replace(',', '').replace('万元', '').replace('人民币', '').replace('元', ''))
                    except Exception:
                        continue
                    if ttt > 20000:
                        ttt = ttt/10000
                    new_h.append(ttt)
                else:
                    continue
            if len(new_h) == 0:
                continue
            temp_single = {}
            a = '_'.join(h)
            # 抽取时间和money
            t = TimeFinder()
            time_all = t.find_time(a)
            if time_all == None:
                continue
            rgrq = time_all[0]
            cpqxr = time_all[0]
            if len(time_all) > 1:
                try:
                    cpdxr = time_all[1]
                    # 相减
                    d1 = datetime.datetime.strptime(cpqxr, '%Y-%m-%d')
                    d2 = datetime.datetime.strptime(cpdxr, '%Y-%m-%d')
                    d = d2 - d1
                    cpqx = str(d.days) + '天'
                except Exception:
                    cpdxr = np.nan
                    cpqx = np.nan
            else:
                cpdxr = np.nan
                cpqx = np.nan
                
            # 筛选出除开数字与包含时间的列
            # 末尾是
            last_two = ['公司', '银行', '信托', '证券',  '分行', '支行', '中心', '业部', '商行', '建行']
            mowei = np.nan
            selected_bank_and_works = []
            for l in h:
                new_l = list(str(l))
                new_l_test = ''.join(l[-2:])
                if new_l_test in last_two:
                    mowei = l
                    continue
                if '资金' in l:
                    continue
                if '收益' in l:
                    continue
                if '到期' in l:
                    continue
                if ',' in l:
                    continue
                if '.' in l:
                    continue
                if '/' in l:
                    continue
                if '年' in l:
                    continue
                if '-' in l:
                    continue
                if len(l) < 4:
                    continue
                if is_number(l):
                    continue
                selected_bank_and_works.append(l)
            if len(selected_bank_and_works) < 1:
                continue
            
            temp_single['认购日期'] = rgrq
            temp_single['产品起息日'] = cpqxr
            temp_single['产品到期日'] = cpdxr
            temp_single['产品期限'] = cpqx
            temp_single['认购金额(万元)'] = new_h[0]
            temp_single['产品发行方名称'] = mowei
            temp_single['理财产品名称'] = selected_bank_and_works[0]
            temp_df.append(temp_single)
        new_df.append(temp_df)
    return new_df

val_contain_date = get_list_data(val_df_tabel)
# test_contain_data = get_list_data(test_df_tabel) 

#### 5.汇总整理数据

In [82]:
# # 将前面提取到的数据整理成对应格式
# sample_id_list = []
# rgrq_list = []
# lccp_list = []
# cpfxf_list = []
# rgje_list = []
# cpqxr_list = []
# cpdxr_list = []
# cpqx_list = []
# sjgmgsmc_list = []
# ggrq_list = []

# sample_id = list(val_df['sample_id'])
# gg = list(val_gm)
# time = list(val_time)
# for i, value in enumerate(sample_id):
#     for j in val_contain_date[i]:
#         sample_id_list.append(sample_id[i])
#         rgrq_list.append(j['认购日期'])
#         lccp_list.append(j['理财产品名称'])
#         cpfxf_list.append(j['产品发行方名称'])
#         rgje_list.append(j['认购金额(万元)'])
#         cpqxr_list.append(j['产品起息日'])
#         cpdxr_list.append(j['产品到期日'])
#         cpqx_list.append(j['产品期限'])
#         sjgmgsmc_list.append(gg[i])
#         ggrq_list.append(time[i])

# result = pd.DataFrame()
# result['sample_id'] = sample_id_list
# result['认购日期'] = rgrq_list
# result['理财产品名称'] = lccp_list
# result['产品发行方名称'] = cpfxf_list
# result['认购金额(万元)'] = rgje_list
# result['产品起息日'] = cpqxr_list
# result['产品到期日'] = cpdxr_list
# result['产品期限'] = cpqx_list
# result['实际购买公司名称'] = sjgmgsmc_list
# result['公告日期'] = ggrq_list
# val_result = result

sample_id_list = []
rgrq_list = []
lccp_list = []
cpfxf_list = []
rgje_list = []
cpqxr_list = []
cpdxr_list = []
cpqx_list = []
sjgmgsmc_list = []
ggrq_list = []

sample_id = list(test_df['sample_id'])
gg = list(test_gm)
time = list(test_time)
for i, value in enumerate(sample_id):
    for j in test_contain_data[i]:
        sample_id_list.append(sample_id[i])
        rgrq_list.append(j['认购日期'])
        lccp_list.append(j['理财产品名称'])
        cpfxf_list.append(j['产品发行方名称'])
        rgje_list.append(j['认购金额(万元)'])
        cpqxr_list.append(j['产品起息日'])
        cpdxr_list.append(j['产品到期日'])
        cpqx_list.append(j['产品期限'])
        sjgmgsmc_list.append(gg[i])
        ggrq_list.append(time[i])

result = pd.DataFrame()
result['sample_id'] = sample_id_list
result['认购日期'] = rgrq_list
result['理财产品名称'] = lccp_list
result['产品发行方名称'] = cpfxf_list
result['认购金额(万元)'] = rgje_list
result['产品起息日'] = cpqxr_list
result['产品到期日'] = cpdxr_list
result['产品期限'] = cpqx_list
result['实际购买公司名称'] = sjgmgsmc_list
result['公告日期'] = ggrq_list
test_result = result
test_result

In [87]:
  def get_F1(val_pred, val_true):
      val_pred = list(val_pred)
      val_true = list(val_true)
      curr = list(set(val_pred).intersection(set(val_true)))
      R = len(curr)/len(val_true)
      P = len(curr)/len(val_pred)
      return 2*P*R/(P+R)

  r = pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left')
  r
  val_true = r['sample_id'].astype(str)  + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str)  +r['产品发行方名称'].astype(str)

  r = temp_single
  r.fillna("").reset_index(drop=True)
  i=0
  i_list=[]
  for index in r.index:
      if r.loc[index].dropna().shape[0]<=4:
        i_list.append(i)
      i+=1
  r.drop(i_list)
  r=drop_judge(r)
  val_pred = r['sample_id'].astype(str)  + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str)  +r['产品发行方名称'].astype(str)

  score = get_F1(val_pred, val_true)
  score

,sample_id,认购日期,理财产品名称,产品发行方名称,理财类型,认购金额(万元),产品起息日,产品到息日,产品期限,资金来源,实际购买公司名称,实际购买公司和上市公司关系,买卖方是否有关联关系,公告日期
0,1739,2017-08-11,中银保本理财-人民币按期开放理财产品,中国银行双流安福街支行,银行理财产品,10000.0,2017-08-11,2017-12-29,140天,自有资金,成都康弘制药有限公司,控股参股公司,否,2017-08-29
1,1739,2017-08-11,中银保本理财-人民币按期开放理财产品,中国银行双流安福街支行,银行理财产品,6000.0,2017-08-11,2017-09-29,49天,自有资金,成都康弘制药有限公司,控股参股公司,否,2017-08-29
2,1806,2017-08-28,结构性存款,上海浦东发展银行股份有限公司北京分行,结构性存款,10000.0,2017-08-28,2017-11-27,91天,自有资金,联动优势电子商务有限公司,控股参股公司,否,2017-08-29
3,7032,2018-04-04,结构性存款,中国光大银行牛市口支行,结构性存款,10000.0,2018-04-04,2018-07-03,90天,自筹资金,四川广安爱众股份有限公司,公司本身,否,2018-04-09
4,2974,2017-08-18,中国建设银行河南分行“乾元”保本型2017年第41期理财产品,建设银行股份有限公司洛阳分行,银行理财产品,13500.0,2017-07-21,2018-01-18,181天,自筹资金,普莱柯生物工程股份有限公司,公司本身,否,2017-08-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5658,3359,2017-08-23,兴业银行“金雪球-优悦”开放式人民币理财产品,兴业银行股份有限公司武侯祠支行,银行理财产品,2500.0,2017-08-23,2017-11-22,91天,"自筹资金,自有资金",四川海特高新技术股份有限公司,公司本身,否,2018-03-27
5659,8553,2018-03-06,蕴通财富·日增利71天,交通银行股份有限公司烟台蓬莱支行,银行理财产品,4200.0,NaT,NaT,71天,自筹资金,烟台东诚药业集团股份有限公司,公司本身,否,2018-03-06
5660,8553,2018-03-06,“蕴通财富·日增利”S款,交通银行股份有限公司烟台蓬莱支行,银行理财产品,1800.0,NaT,NaT,NaN,自筹资金,烟台东诚药业集团股份有限公司,公司本身,否,2018-03-06
5661,8553,2018-03-06,中银保本理财-人民币全球智选【CNYQQZX】,中国银行股份有限公司上海市闵行支行营业部,银行理财产品,2000.0,NaT,NaT,92天,自有资金,上海欣科医药有限公司,控股参股公司,否,2018-03-06


,认购日期,产品起息日,产品到息日,产品期限,认购金额(万元),产品发行方名称,理财产品名称,sample_id
0,2017-08-28,2017-08-28,2017-11-27,91天,10000,,银行,1806
1,2019-04-16,2019-04-16,2019-07-18,93天,4000,中国银行股份有限公司厦门杏林支行,中银平稳理财计划-智荟系列,3581
2,2019-04-16,2019-04-16,2019-07-16,91天,3000,厦门银行股份有限公司杏林支行,结构性存款,3581
3,2019-04-17,2019-04-17,2019-07-15,89天,3000,兴证证券资产管理有限公司,兴证资管鑫利5号集合资产管理计划,3581
4,2019-04-16,2019-04-16,2019-07-16,91天,10000,兴业银行股份有限,兴业银行“金雪球-优悦”,3581
...,...,...,...,...,...,...,...,...
7072,,,,回天,,,,193
7073,,,,回天,,,,193
7074,2018-05-08,2018-05-08,,天,800,中国农业银行,安心快线天天利滚利第2期,193
7075,,,,回天,,,,193


,认购日期,产品起息日,产品到息日,产品期限,认购金额(万元),产品发行方名称,理财产品名称,sample_id
0,2017-08-28,2017-08-28,2017-11-27,91天,10000,NaN,银行,1806
1,2019-04-16,2019-04-16,2019-07-18,93天,4000,中国银行股份有限公司厦门杏林支行,中银平稳理财计划-智荟系列,3581
2,2019-04-16,2019-04-16,2019-07-16,91天,3000,厦门银行股份有限公司杏林支行,结构性存款,3581
3,2019-04-17,2019-04-17,2019-07-15,89天,3000,兴证证券资产管理有限公司,兴证资管鑫利5号集合资产管理计划,3581
4,2019-04-16,2019-04-16,2019-07-16,91天,10000,兴业银行股份有限,兴业银行“金雪球-优悦”,3581
...,...,...,...,...,...,...,...,...
7072,NaN,NaN,NaN,回天,,,,193
7073,NaN,NaN,NaN,回天,,,,193
7074,2018-05-08,2018-05-08,NaN,天,800,中国农业银行,安心快线天天利滚利第2期,193
7075,NaN,NaN,NaN,回天,,,,193


2%|▏         | 116/7077 [00:00<00:30, 224.65it/s]


TypeError: object of type 'float' has no len()

In [85]:
def drop_judge(result):
    global judge_title_result
    score_limit=30
    drop_list=[]
    index=0
    for sample_id,product_name in tqdm(result[["sample_id","理财产品名称"]].values):
        score_list=[]
        for text in judge_title_result[judge_title_result["sample_id"]==sample_id][4].values:
            score_list.append(fuzz.partial_ratio(product_name,text))
        if np.max(pd.DataFrame(score_list)[0])<=score_limit:
            drop_list.append(index)
        index+=1 
    
    return result.copy().drop(drop_list)

  def get_F1(val_pred, val_true):
      val_pred = list(val_pred)
      val_true = list(val_true)
      curr = list(set(val_pred).intersection(set(val_true)))
      R = len(curr)/len(val_true)
      P = len(curr)/len(val_pred)
      return 2*P*R/(P+R)

  r = pd.merge(val_df[['sample_id']], train_outputs, on='sample_id', how='left')
  val_true = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到息日'].astype(str) + r['产品期限'].astype(str) +r['产品发行方名称'].astype(str)

  r = drop_judge(result)
  print(r.shape)
  val_pred = r['sample_id'].astype(str) + r['认购日期'].astype(str) + r['理财产品名称'].astype(str) + r['认购金额(万元)'].astype(str) + r['产品起息日'].astype(str)+ r['产品到期日'].astype(str) + r['产品期限'].astype(str) +r['产品发行方名称'].astype(str)

  score = get_F1(val_pred, val_true)
  score

val_result_file=val_result.sort_values(by='sample_id').reset_index(drop=True)
val_true_file=pd.merge(train_outputs, val_df['sample_id'], on='sample_id',how="right").sort_values(by="sample_id").reset_index(drop=True)
# train_outputs
# val_result_file
# val_true_file
val_id_list=val_result_file["sample_id"].unique()
val_id_count={}
val_id_count["sample_id"]=[]
val_id_count["预测"]=[]
val_id_count["实际"]=[]
for item in val_id_list:
    val_id_count["sample_id"].append(item)
    val_id_count["预测"].append(val_result_file[val_result_file["sample_id"]==item].shape[0])
    val_id_count["实际"].append(val_true_file[val_true_file["sample_id"]==item].shape[0])

val_id_count=pd.DataFrame(val_id_count)
val_id_count["差值"]=val_id_count["预测"]-val_id_count["实际"]
print(val_id_count[val_id_count["差值"]==0].shape[0]/val_id_count.shape[0])
val_id_count.to_excel("验证集row数量对比.xlsx")